<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula14_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autokeras

In [1]:
!pip install autokeras -q

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak


In [29]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)


(60000, 28, 28)
(60000,)
[5 0 4]


In [30]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=2)
# Feed the image classifier with training data.
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=3,
)

# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


Trial 2 Complete [00h 27m 57s]
val_loss: 0.3146403431892395

Best val_loss So Far: 0.045711446553468704
Total elapsed time: 00h 28m 43s
Epoch 1/3
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1598 - accuracy: 0.9515
Epoch 2/3
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0713 - accuracy: 0.9782
Epoch 3/3
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0592 - accuracy: 0.9818


313/313 [==============================] - 1s 2ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]
313/313 [==============================] - 1s 3ms/step - loss: 0.0344 - accuracy: 0.9893
[0.03439457714557648, 0.989300012588501]


#Keras Tuner

In [5]:
!pip install keras-tuner -q

In [16]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
  units = hp.Int(name="units", min_value=16, max_value=64, step=16)
  model = keras.Sequential([
      layers.Dense(units, activation="relu"),
      layers.Dense(10, activation="softmax")
  ])
  optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
  model.compile(
      optimizer=optimizer,
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"])
  return model

In [10]:
!pip install keras-tuner[bayesian] -q

In [17]:
import keras_tuner as kt

tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [18]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [19]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_val,y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.9638499915599823

Best val_accuracy So Far: 0.9717999994754791
Total elapsed time: 00h 02m 51s


In [20]:
top_n = 3
best_hps = tuner.get_best_hyperparameters(top_n)
best_hps

In [23]:
def get_best_epoch(hp):
  model = build_model(hp)
  callbacks = [keras.callbacks.EarlyStopping(
      monitor="val_loss", mode="min", patience=10)]
  history = model.fit(
      x_train, y_train,
      validation_data=(x_val,y_val),
      epochs=20,
      batch_size=128,
      callbacks=callbacks)
  val_loss_per_epoch = history.history["val_loss"]
  best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
  print(f"Best epoch: {best_epoch}")
  return best_epoch

In [26]:
def get_best_trained_model(hp):
  best_epoch = get_best_epoch(hp)
  model = build_model(hp)
  model.fit(x_train_full, y_train_full,
            batch_size=128, epochs=int(best_epoch * 1.2))
  return model

best_models = []
for hp in best_hps:
  model = get_best_trained_model(hp)
  print("\n\n### Evaluate ###\n")
  evaluate = model.evaluate(x_test, y_test)
  print(evaluate)
  print("### ### ### ###")
  best_models.append(model)

Epoch 1/20
391/391 [==============================] - 3s 5ms/step - loss: 0.4157 - accuracy: 0.8881 - val_loss: 0.2301 - val_accuracy: 0.9350
Epoch 2/20
391/391 [==============================] - 2s 4ms/step - loss: 0.2189 - accuracy: 0.9371 - val_loss: 0.1787 - val_accuracy: 0.9517
Epoch 3/20
391/391 [==============================] - 1s 4ms/step - loss: 0.1720 - accuracy: 0.9506 - val_loss: 0.1535 - val_accuracy: 0.9576
Epoch 4/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1419 - accuracy: 0.9581 - val_loss: 0.1400 - val_accuracy: 0.9601
Epoch 5/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1210 - accuracy: 0.9659 - val_loss: 0.1302 - val_accuracy: 0.9620
Epoch 6/20
391/391 [==============================] - 1s 3ms/step - loss: 0.1053 - accuracy: 0.9688 - val_loss: 0.1195 - val_accuracy: 0.9654
Epoch 7/20
391/391 [==============================] - 1s 4ms/step - loss: 0.0929 - accuracy: 0.9734 - val_loss: 0.1141 - val_accuracy: 0.9670
Epoch 